Import libraries

In [3]:
import pandas as pd 
import numpy as np 

Create DataFrame

In [5]:
df = pd.DataFrame([[1,2,3],[4,5,6],[7,8,9]], columns=["A", "B", "C"])

In [ ]:
df.head()


In [ ]:
df.tail()

In [ ]:
df.columns

In [ ]:
df.index.to_list()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.shape #num of rows and columns

In [ ]:
df.info() #memory usage

In [ ]:
df.size #total number of items

Loading dataframes from files

In [4]:
#Load csv file

coffee = pd.read_csv('./warmup-data/coffee.csv')

bios = pd.read_csv('./data/bios.csv')

In [ ]:
coffee.head()

In [28]:
#Load parquet file

results = pd.read_parquet('./data/results.parquet')

In [ ]:
results.head()

In [30]:
#Load excel file

# olympics_data = pd.read_excel('./data/olympics-data.xlsx')

#including sheet_name
olympics_data = pd.read_excel('./data/olympics-data.xlsx', sheet_name="results")

In [ ]:
olympics_data.head()

Assessing data with pandas

In [ ]:

# coffee
# print(coffee)
# display(coffee)
coffee.sample(10) #random values
# coffee.sample(10, random_state=1) #deterministic

In [ ]:
#Alter index
# coffee.index = coffee["Day"]
# coffee.loc["Monday":"Wednesday", "Units Sold"]

In [ ]:
#Access specific rows and columns

#loc
# coffee.loc[[0,1,2]]

#Slice
# coffee.loc[5:12]

#Slice by column name
# coffee.loc[5:12, "Day"]
# coffee.loc[5:12, ["Day", "Units Sold"]]

#iloc #only using index values
coffee.iloc[5:12, [0,2]]

In [ ]:
#Update a record using loc #Latte Units Sold
coffee.loc[1, "Units Sold"] = 10
coffee.head()

In [ ]:
coffee.at[0,"Units Sold"] #only single values

In [ ]:
coffee.iat[0,0] #only single values

In [ ]:
coffee["Day"]

In [ ]:
# coffee.sort_values("Units Sold", ascending=False) #defaults as ASC order
coffee.sort_values(["Units Sold", "Coffee Type"], ascending=[0,1])

In [ ]:
for index, row in coffee.iterrows():
    print(index)
    print(row)
    print("\n\n")
    
#does have a neg performance impact

Filtering data

In [ ]:

bios.head(2)

In [ ]:
#Lets look at height and weight
bios.info()

#height and weight are floats

In [ ]:
bios.loc[bios["height_cm"] > 215, ['name', 'height_cm']]

In [ ]:
bios[(bios["height_cm"] > 215) & (bios['born_country']=='AUS')]

In [ ]:
# bios[bios['name'].str.contains("Luc")] #case sensitive
bios[bios['name'].str.contains("luc|patrick", case=False)] #regex

Regex

In [ ]:

#Find all athletes born in cities that start with a vowel
vowel_cities = bios[bios['born_city'].str.contains(r'^[AEIOUaeiou]', na=False)]
vowel_cities.head(2)


In [ ]:
#Find athletes names ending in 'son'
son_names = bios[bios['name'].str.contains(r'son$', case=False, na=False)]
son_names.head(2)

In [ ]:
#Find athletes with names that do not contain a vowel
no_vowels = bios[bios['name'].str.contains(r'^[AEIOUaeiou]*$', na=False)]
no_vowels

In [ ]:
#Find athletes whose names contain a hyphen or an apostrophe
hyphen_apostrophe = bios[bios['name'].str.contains(r"[-']", na=False)]
hyphen_apostrophe.head(2)

In [ ]:
#isin
# bios[bios["born_country"].isin(["FRA", "ABC", "AUS"])] #Added non country to see what the impact would be
bios[bios["born_country"].isin(["FRA", "ABC", "AUS"]) & (bios['name'].str.startswith("Bob"))]

In [ ]:
#Another way of filtering results
bios.query('born_country == "AUS" and born_city == "Sydney"')

Adding and removing columns

In [ ]:

# coffee.head(2)
coffee["Price"] = 5.99 #all coffee types will have 5.99 as price
coffee.head(2)

coffee["Price"] = np.where(coffee["Coffee Type"]=="Espresso", 4.25, 5.99)
coffee.head(2)

In [ ]:
#Drop column
# coffee.drop(columns=['Price'], inplace=True)
coffee.head(2)

In [ ]:
coffee = coffee.rename(columns={'New_Price': 'Price'}) # or can use inplace=True without resetting (coffee=)
coffee.head(2)

In [ ]:
coffee['Revenue'] = coffee['Units Sold'] * coffee['Price']
coffee.head(2)

In [ ]:
bios.head(2)

Create a copy of the source data

In [4]:
bios_new = bios.copy()

In [5]:
bios_new['first_name'] = bios_new['name'].str.split(' ').str[0]

In [ ]:
bios_new.head(2)
bios_new.query('first_name == "Keith"')

Convert born_date to date data type -> heaps of format methods for date data type look up

In [ ]:
# bios_new.info()
bios_new['born_datetime'] = pd.to_datetime(bios_new['born_date'])
bios_new.head(2)
bios_new.info()

In [ ]:
#Create "year" column from "born_datetime" column, now  "born_datetime" has been created as datetime data type
bios_new['born_year'] = bios_new['born_datetime'].dt.year

bios_new[['name', 'born_year']]

Save new file to csv

In [14]:
bios_new.to_csv('./data/bios_new.csv', index=False)

Create custom columns

In [ ]:
bios['height_category'] = bios['height_cm'].apply(lambda x: 'short' if x <165 else ('average' if x < 185 else 'tall'))
bios.head(2)


Create a function to populate a new column

In [5]:
def categorize_athlete(row):
    if row['height_cm'] <175 and row['weight_kg'] <70:
        return 'Lightweight'
    elif row['height_cm'] <180 and row['weight_kg'] <80:
        return 'Middleweight'
    
    else:
        return 'Heavyweight'
    
bios['Category'] = bios.apply(categorize_athlete, axis=1)

In [ ]:
bios.head(2)

Merge and concatenating data

In [ ]:
nocs = pd.read_csv('./data/noc_regions.csv')
nocs.head(2)

In [ ]:
bios = pd.merge(bios, nocs, left_on='born_country', right_on='NOC', how='left', suffixes=["bios", "nocdf"]) #how= same concept as sql joins
bios.head(2)

In [ ]:
bios.rename(columns={'region': 'born_country_full'}, inplace=True)
bios.head(2)

In [19]:
bios = bios.drop(['NOC_x','NOC_y'],axis=1)
bios.head(2)

In [ ]:
bios['NOCbios'] != bios['born_country_full']
bios

Create new dfs and then concat

In [24]:
aus = bios[bios['born_country']=='AUS'].copy()
gbr = bios[bios['born_country']=='GBR'].copy()

In [ ]:
aus_gbr = pd.concat([aus,gbr])
aus_gbr.tail(2)

Using merge

In [ ]:
combined_df = pd.merge(results, bios, on='athlete_id', how='left')
combined_df.head(2)

Handling null values

In [ ]:
#For the purpose of study, adding null values into dataframe
coffee.loc[[0,1], 'Units Sold'] = np.nan
coffee.head()


In [ ]:
#Check for null|na in dataframe
coffee.info()
coffee.isna().sum()

In [ ]:
coffee = coffee.fillna(coffee['Units Sold'].mean())
#coffee = coffee.fillna(coffee['Units Sold'].interpolate()) #applies values based on the pattern of the values in the dataframe.

coffee

In [ ]:
#Drop entire row where there is a null|na
coffee.dropna()

#coffee.dropna(subset=['Units Sold'], inplace=True)

In [ ]:
#Find rows where null|na found in a column
coffee[coffee['Units Sold'].isna()]

In [ ]:
#Find rows where there is no null|na found in a column
coffee[coffee['Units Sold'].notna()]

Aggregating data

In [ ]:
bios['born_city'].value_counts()

In [ ]:
bios[bios['born_country']=='AUS']['born_region'].value_counts() #.tail()

In [ ]:
bios['born_date'] = pd.to_datetime(bios['born_date'])

#Most common birth year
bios.groupby(bios['born_date'].dt.year)['name'].count().reset_index().sort_values('name', ascending=False)

In [ ]:
coffee.groupby(['Coffee Type'])['Units Sold'].mean()

In [ ]:
coffee

In [ ]:
coffee.groupby(['Coffee Type']).agg({'Units Sold': 'sum', 'Price': 'mean'})

In [ ]:
coffee.groupby(['Coffee Type', 'Day']).agg({'Units Sold': 'sum', 'Price': 'mean'})

Pivot table

In [ ]:
pivot = coffee.pivot(columns='Coffee Type', index='Day', values='Revenue')
pivot

In [ ]:
pivot.loc['Monday','Latte']

In [ ]:
pivot.sum()

In [ ]:
pivot.sum(axis=1)

Advanced Funtionality

In [ ]:
#Shift -> comparring data .... similar to lead | lag???
coffee['yesterday_revenue'] = coffee['Revenue'].shift(2) # can use .shift(-2) as well
coffee

In [ ]:
coffee['pct_change'] = coffee['Revenue'] / coffee['yesterday_revenue']
coffee

In [31]:
#Compare weights
bios['height_rank'] = bios['height_cm'].rank()

In [ ]:
bios.sort_values(['height_rank'],ascending=False)
bios

In [ ]:
bios.sort_values(['height_rank']).sample(10)[['name', 'height_rank']]
bios

In [ ]:
#Cumlative revenue
coffee['cumlative_revenue'] = coffee['Revenue'].cumsum()
coffee.head()

In [ ]:
#Cumlative revenue last 3 days for latte
latte = coffee[coffee['Coffee Type']=="Latte"].copy()
latte['3day'] = latte['Units Sold'].rolling(3).sum()
latte

New funtionality

In [12]:
pd.__version__

'2.2.2'

In [13]:
results_numpy = pd.read_csv('./data/results.csv')
results_arrow = pd.read_csv('./data/results.csv', engine='pyarrow', dtype_backend='pyarrow')

In [14]:
results_numpy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308408 entries, 0 to 308407
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   year        305807 non-null  float64
 1   type        305807 non-null  object 
 2   discipline  308407 non-null  object 
 3   event       308408 non-null  object 
 4   as          308408 non-null  object 
 5   athlete_id  308408 non-null  int64  
 6   noc         308407 non-null  object 
 7   team        121714 non-null  object 
 8   place       283193 non-null  float64
 9   tied        308408 non-null  bool   
 10  medal       44139 non-null   object 
dtypes: bool(1), float64(2), int64(1), object(7)
memory usage: 23.8+ MB


In [15]:
results_arrow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308408 entries, 0 to 308407
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype          
---  ------      --------------   -----          
 0   year        305807 non-null  double[pyarrow]
 1   type        305807 non-null  string[pyarrow]
 2   discipline  308407 non-null  string[pyarrow]
 3   event       308408 non-null  string[pyarrow]
 4   as          308408 non-null  string[pyarrow]
 5   athlete_id  308408 non-null  int64[pyarrow] 
 6   noc         308407 non-null  string[pyarrow]
 7   team        121714 non-null  string[pyarrow]
 8   place       283193 non-null  double[pyarrow]
 9   tied        308408 non-null  bool[pyarrow]  
 10  medal       44139 non-null   string[pyarrow]
dtypes: bool[pyarrow](1), double[pyarrow](2), int64[pyarrow](1), string[pyarrow](7)
memory usage: 37.5 MB
